In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex

data: DiffResult = load_pkl("src")


ISSUES = {
    IssueEnum.OLD: r"\ \ \ # \ \ ",
    IssueEnum.ADD: r"$+$\tiny%",
    IssueEnum.REMOVE: r"$-$\tiny%",
    IssueEnum.CHANGE: r"$\Delta$\tiny%",
}

GROUPS = {
    VersionGroup.LTS: "LTS (2 yr)",
    VersionGroup.REGULAR: "Regular Releases (6 mo)",
}


results = {}

def format_val(kind, val):
    if kind == DepKind.FUNC:
        return f"{val / 1000:.0f}k"
    elif kind == DepKind.STRUCT:
        return f"{val / 1000:.1f}k"
    return val

for (group, pair), diffs in data.items():
    if group not in GROUPS:
        continue

    result = {}
    old = 0
    for (kind, key), s in diffs.items():
        if kind == DepKind.SYSCALL:
            continue

        if key not in ISSUES.keys():
            continue

        display_key = ISSUES[key]

        if key == "Old":
            old = s
            s = format_val(kind, s)
        else:
            if s == 0:
                s = GRAY_DASH
            else:
                s = f"{s / old * 100:.0f}"
            s = r"\multirow{2}{*}{" + str(s) + "}"
        result[(kind, display_key)] = s

    v1 = group.to_str(pair.v1)
    v2 = group.to_str(pair.v2)

    group_name = GROUPS[group]
    results[(group_name, v1)] = result
    if pair.v2 == list(group)[-1]:
        result = {k: "" for k in result.keys()}
        for (kind, key) in result.keys():
            if key == ISSUES[IssueEnum.OLD]:
                result[(kind, key)] = format_val(kind, diffs[kind, IssueEnum.NEW])
        results[(group_name, v2)] = result


df = pd.DataFrame(results).T

latex = df.to_latex(multicolumn_format="c|", column_format=r"cc|rrrr|rrrr|rrrr|rrrr")
save_latex(latex, "src")
df

[          pkl.py:18 ] INFO: Loding src from /Users/szhong/Downloads/bpf-study/output/src.pkl
[        latex.py:68 ] INFO: Saved src to /Users/szhong/Downloads/bpf-study/paper/tabs/src.tex


Function                                                                     Struct                                                                 Tracepoint                                                                                      LSM Hook                                                                                                                     
                             \ \ \ # \ \             $+$\tiny%            $-$\tiny%      $\Delta$\tiny% \ \ \ # \ \             $+$\tiny%           $-$\tiny%       $\Delta$\tiny% \ \ \ # \ \             $+$\tiny%                              $-$\tiny%       $\Delta$\tiny% \ \ \ # \ \                               $+$\tiny%                              $-$\tiny%                         $\Delta$\tiny%
LTS (2 yr)              4.4           36k  \multirow{2}{*}{24}   \multirow{2}{*}{7}  \multirow{2}{*}{5}         6.2k  \multirow{2}{*}{24}  \multirow{2}{*}{4}  \multirow{2}{*}{18}          502  \multirow{2}{*}{39}                     \multirow{2}{*}{5}   \multirow{2}{*}{8}          168                     \multirow{2}{*}{9}                     \multirow{2}{*}{4}                    \multirow{2}{*}{11}
                        4.15          42k  \multirow{2}{*}{21}   \multirow{2}{*}{8}  \multirow{2}{*}{4}         7.3k  \multirow{2}{*}{19}  \multirow{2}{*}{4}  \multirow{2}{*}{15}          675  \multirow{2}{*}{15}                     \multirow{2}{*}{3}   \multirow{2}{*}{8}          177                     \multirow{2}{*}{8}                     \multirow{2}{*}{1}                    \multirow{2}{*}{10}
                        5.4           48k  \multirow{2}{*}{22}  \multirow{2}{*}{10}  \multirow{2}{*}{5}         8.4k  \multirow{2}{*}{17}  \multirow{2}{*}{6}  \multirow{2}{*}{16}          752  \multirow{2}{*}{14}                     \multirow{2}{*}{5}  \multirow{2}{*}{16}          189                     \multirow{2}{*}{8}                     \multirow{2}{*}{1}                    \multirow{2}{*}{14}
                        5.15          54k  \multirow{2}{*}{23}   \multirow{2}{*}{8}  \multirow{2}{*}{6}         9.3k  \multirow{2}{*}{16}  \multirow{2}{*}{4}  \multirow{2}{*}{16}          818  \multirow{2}{*}{18}                     \multirow{2}{*}{4}  \multirow{2}{*}{14}          203                    \multirow{2}{*}{10}                     \multirow{2}{*}{2}                    \multirow{2}{*}{13}
                        6.8           62k                                                                      10.5k                                                                        932                                                                                           219                                                                                                                     
Regular Releases (6 mo) 4.4           36k   \multirow{2}{*}{8}   \multirow{2}{*}{3}  \multirow{2}{*}{2}         6.2k   \multirow{2}{*}{9}  \multirow{2}{*}{2}   \multirow{2}{*}{9}          502   \multirow{2}{*}{7}  \multirow{2}{*}{\color{lightgray}{-}}   \multirow{2}{*}{5}          168                     \multirow{2}{*}{2}                     \multirow{2}{*}{2}                    \multirow{2}{*}{10}
                        4.8           38k   \multirow{2}{*}{4}   \multirow{2}{*}{2}  \multirow{2}{*}{1}         6.6k   \multirow{2}{*}{3}  \multirow{2}{*}{1}   \multirow{2}{*}{6}          539   \multirow{2}{*}{5}                     \multirow{2}{*}{1}   \multirow{2}{*}{3}          168                     \multirow{2}{*}{2}  \multirow{2}{*}{\color{lightgray}{-}}  \multirow{2}{*}{\color{lightgray}{-}}
                        4.10          39k   \multirow{2}{*}{7}   \multirow{2}{*}{3}  \multirow{2}{*}{2}         6.8k   \multirow{2}{*}{5}  \multirow{2}{*}{1}   \multirow{2}{*}{9}          559  \multirow{2}{*}{15}                     \multirow{2}{*}{1}   \multirow{2}{*}{2}          171                     \multirow{2}{*}{4}                     \multirow{2}{*}{1}                     \multirow{2}{*}{2}
   

In [13]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex

data: DiffResult = load_pkl("src")


def multicolumn(s, n=2, format="c"):
    return r"\multicolumn{2}{ " + format + r"}{" + s + "}"


KINDS = {
    DepKind.FUNC: DepKind.FUNC,
    DepKind.STRUCT: DepKind.STRUCT,
    DepKind.TRACEPOINT: "Tracept",
}

results = {}
for (group, pair), diffs in data.items():
    if group not in [VersionGroup.LTS, VersionGroup.FIRST_LAST]:
        continue

    result = {}
    change = 0
    for (kind, key), v in diffs.items():
        if v == 0:
            continue
        if key in (IssueEnum.ADD, IssueEnum.REMOVE, IssueEnum.OLD, IssueEnum.NEW):
            continue
        if kind == DepKind.TRACEPOINT and key not in (
            IssueEnum.CHANGE,
            IssueEnum.TRACE_EVENT_CHANGE,
            IssueEnum.TRACE_FUNC_CHANGE,
        ):
            continue
        kind = KINDS.get(kind)
        if kind is None:
            continue

        if key == IssueEnum.CHANGE:
            change = v
            key = "No. changed"
            v = f"{v / 1000:.1f}k" if v > 1000 else str(v)
        else:
            key = f" - {key}"
            v = f"{v / change * 100:.0f}" + r"%"
        
        result[(kind, key)] = v

    v1 = group.to_str(pair.v1)
    v2 = group.to_str(pair.v2)

    def header(v):
        return ("Linux Kernel Version", v)

    # if pair.v1 == list(group)[0]:
    #     results[header(v1)] = {k: "" for k in result.keys()}
    results[("Linux Kernel Version", v1, v2)] = result

df = pd.DataFrame(results)
df = df.fillna(GRAY_DASH)
df.columns.names = ["", r"\multicolumn{1}{r|}{From}", r"\multicolumn{1}{r|}{To}"]
# df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: x + " &")
latex = df.to_latex(multicolumn_format="c", column_format="cl|rrrr|r")
# latex = latex.replace(
#     r"\multicolumn{5}{c}{Linux Kernel Version}",
#     r"\multicolumn{6}{c}{Linux Kernel Version}",
# )
# latex = latex.replace(
#     "4.4 & 4.15 & 5.4 & 5.15 & 6.8",
#     r"\multicolumn{6}{c}{4.4 \quad 4.15 \quad 5.4 \quad 5.15 \quad 6.8}",
# )
save_latex(latex, "breakdown")
df

[          pkl.py:18 ] INFO: Loding src from /Users/szhong/Downloads/bpf-study/output/src.pkl
[        latex.py:57 ] INFO: Saved breakdown to /Users/szhong/Downloads/bpf-study/paper/tabs/breakdown.tex


Linux Kernel Version                        
\multicolumn{1}{r|}{From}                        4.4  4.15   5.4  5.15   4.4
\multicolumn{1}{r|}{To}                         4.15   5.4  5.15   6.8   6.8
Function No. changed                            2.0k  1.8k  2.2k  3.0k  4.7k
          - Param added                          57%   53%   60%   51%   53%
          - Param removed                        41%   36%   44%   48%   42%
          - Param reordered                      19%   19%   25%   19%   19%
          - Param type changed                   26%   23%   26%   25%   30%
          - Return type changed                  15%   21%   13%   17%   19%
Struct   No. changed                            1.1k  1.1k  1.3k  1.5k  1.9k
          - Field added                          72%   74%   75%   74%   76%
          - Field removed                        41%   40%   40%   42%   49%
          - Field type changed                   37%   34%   32%   34%   44%
Tracept  No. changed                              39    54   119   115   131
          - Event changed                        95%   81%   86%   92%   85%
          - Func changed                         46%   54%   32%   51%   62%

In [4]:
data.keys()

dict_keys([(Versions(LTS), (4.4.0-21-generic-amd64, 4.15.0-20-generic-amd64)), (Versions(LTS), (4.15.0-20-generic-amd64, 5.4.0-26-generic-amd64)), (Versions(LTS), (5.4.0-26-generic-amd64, 5.15.0-25-generic-amd64)), (Versions(LTS), (5.15.0-25-generic-amd64, 6.8.0-31-generic-amd64)), (Versions(Regular), (4.4.0-21-generic-amd64, 4.8.0-22-generic-amd64)), (Versions(Regular), (4.8.0-22-generic-amd64, 4.10.0-19-generic-amd64)), (Versions(Regular), (4.10.0-19-generic-amd64, 4.13.0-16-generic-amd64)), (Versions(Regular), (4.13.0-16-generic-amd64, 4.15.0-20-generic-amd64)), (Versions(Regular), (4.15.0-20-generic-amd64, 4.18.0-10-generic-amd64)), (Versions(Regular), (4.18.0-10-generic-amd64, 5.0.0-13-generic-amd64)), (Versions(Regular), (5.0.0-13-generic-amd64, 5.3.0-18-generic-amd64)), (Versions(Regular), (5.3.0-18-generic-amd64, 5.4.0-26-generic-amd64)), (Versions(Regular), (5.4.0-26-generic-amd64, 5.8.0-25-generic-amd64)), (Versions(Regular), (5.8.0-25-generic-amd64, 5.11.0-16-generic-amd64))